In [1]:
import argparse
import sys
import re
import os, glob, datetime, time
import numpy as np
import torch
import torch.nn as nn
from torch.nn.modules.loss import _Loss
import torch.nn.init as init
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import data_generator as dg
from data_generator import DenoisingDataset
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# Params
if any(["jupyter" in arg for arg in sys.argv]):
    # Simulate command line arguments (replace these with your desired defaults)
    sys.argv = ['ipykernel_launcher.py', '--model', 'DnCNN', '--batch_size', '64', '--train_data', 'data/Train400', '--sigma', '4', '--epoch', '3', '--lr', '0.0005']

parser = argparse.ArgumentParser(description='PyTorch DnCNN')
parser.add_argument('--model', default='DnCNN', type=str, help='choose a type of model')
parser.add_argument('--batch_size', default=64, type=int, help='batch size')
parser.add_argument('--train_data', default='data/Train400', type=str, help='path of train data')
parser.add_argument('--sigma', default=4, type=int, help='noise level')
parser.add_argument('--epoch', default=3, type=int, help='number of train epoches')
parser.add_argument('--lr', default=5e-4, type=float, help='initial learning rate for Adam')
args = parser.parse_args()
batch_size = args.batch_size
cuda = torch.cuda.is_available()
n_epoch = args.epoch
sigma = args.sigma

save_dir = os.path.join('models', args.model+'_' + 'sigma' + str(sigma))

if not os.path.exists(save_dir):
    os.mkdir(save_dir)
        
        
class DnCNN(nn.Module):
    def __init__(self, out_channels=64, in_channels=1, kernel_size=3):
        super(DnCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1, stride=1)
        self.BN = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        output_layers = []
        for _ in range(10):
            for sub in self.make_Residual():
                output_layers.append(sub)
        self.out = nn.Sequential(*output_layers)
        self.res = nn.Conv2d(64, 1, kernel_size=3, padding=1, stride=1)
        
    def make_Residual(self, in_channels=64, out_channels=64, kernel_size=3):
        i = 0
        layers = []
        for i in range(2):
            layers.append(nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=1, bias=True))
            layers.append(nn.BatchNorm2d(64, eps=0.0001, momentum=0.95))
            layers.append(nn.ReLU(inplace=True))
        return layers

    
    def forward(self, x):
        y = x
        out = self.relu(self.BN(self.conv1(x)))
        out = self.out(out)
        dn = y - self.res(out)
        return dn
        

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                print('init weight')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)

                
class sum_squared_error(_Loss):  # PyTorch 0.4.1
    """
    Definition: sum_squared_error = 1/2 * nn.MSELoss(reduction = 'sum')
    The backward is defined as: input-target
    """
    def __init__(self, size_average=None, reduce=None, reduction='sum'):
        super(sum_squared_error, self).__init__(size_average, reduce, reduction)

    def forward(self, input, target):
        # return torch.sum(torch.pow(input-target,2), (0,1,2,3)).div_(2)
        return torch.nn.functional.mse_loss(input, target, size_average=None, reduce=None, reduction='sum').div_(2)


def log(*args, **kwargs):
     print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S:"), *args, **kwargs)

In [4]:
import torch

# Attempt to manually select a CUDA device
try:
    torch.cuda.current_device()
    print("Current CUDA device:", torch.cuda.get_device_name())
except Exception as e:
    print("Error accessing CUDA device:", str(e))

Current CUDA device: NVIDIA GeForce GTX 1080 Ti


In [5]:
from tqdm import tqdm

if __name__ == '__main__':
    # model selection
    print('===> Building model')
    model = DnCNN()
    
    initial_epoch = 0
    model.train()
    # criterion = nn.MSELoss(reduction = 'sum')  # PyTorch 0.4.1
    criterion = sum_squared_error()
    if cuda:
        model = model.cuda()
         # device_ids = [0]
         # model = nn.DataParallel(model, device_ids=device_ids).cuda()
         # criterion = criterion.cuda()
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay = 1e-4)
    scheduler = MultiStepLR(optimizer, milestones=[3, 6, 9], gamma=0.2)  # learning rates
    for epoch in tqdm(range(initial_epoch, n_epoch)):

        scheduler.step(epoch)  # step to the learning rate in this epcoh
        xs = dg.datagenerator(data_dir=args.train_data)
        xs = xs.astype('float32')/255.0
        xs = torch.from_numpy(xs.transpose((0, 3, 1, 2)))  # tensor of the clean patches, NXCXHXW
        DDataset = DenoisingDataset(xs, sigma)
        DLoader = DataLoader(dataset=DDataset, num_workers=4, drop_last=True, batch_size=batch_size, shuffle=True)
        epoch_loss = 0
        start_time = time.time()

        for n_count, batch_yx in enumerate(DLoader):
                optimizer.zero_grad()
                if cuda:
                    batch_x, batch_y = batch_yx[1].cuda(), batch_yx[0].cuda()
                loss = criterion(model(batch_y), batch_x)
                epoch_loss += loss.item()
                loss.backward()
                optimizer.step()
                if n_count % 10 == 0:
                    print('%4d %4d / %4d loss = %2.4f' % (epoch+1, n_count, xs.size(0)//batch_size, loss.item()/batch_size))
        elapsed_time = time.time() - start_time

        log('epcoh = %4d , loss = %4.4f , time = %4.2f s' % (epoch+1, epoch_loss/n_count, elapsed_time))
        np.savetxt('train_result.txt', np.hstack((epoch+1, epoch_loss/n_count, elapsed_time)), fmt='%2.4f')
        # torch.save(model.state_dict(), os.path.join(save_dir, 'model_%03d.pth' % (epoch+1)))
        torch.save(model, os.path.join(save_dir, 'model_%03d.pth' % (epoch+1)))


===> Building model


  0%|          | 0/3 [00:00<?, ?it/s]

^_^-training data finished-^_^
   1    0 / 2728 loss = 4456.5649
   1   10 / 2728 loss = 1448.0883
   1   20 / 2728 loss = 984.8729
   1   30 / 2728 loss = 654.9639
   1   40 / 2728 loss = 374.7516
   1   50 / 2728 loss = 297.6810
   1   60 / 2728 loss = 258.7581
   1   70 / 2728 loss = 224.7777
   1   80 / 2728 loss = 248.0746
   1   90 / 2728 loss = 170.3078
   1  100 / 2728 loss = 152.8036
   1  110 / 2728 loss = 156.1373
   1  120 / 2728 loss = 122.0618
   1  130 / 2728 loss = 102.7961
   1  140 / 2728 loss = 97.9845
   1  150 / 2728 loss = 84.7896
   1  160 / 2728 loss = 92.1766
   1  170 / 2728 loss = 72.6370
   1  180 / 2728 loss = 90.2166
   1  190 / 2728 loss = 62.2101
   1  200 / 2728 loss = 68.6387
   1  210 / 2728 loss = 109.7892
   1  220 / 2728 loss = 62.4841
   1  230 / 2728 loss = 53.5848
   1  240 / 2728 loss = 52.8305
   1  250 / 2728 loss = 68.1986
   1  260 / 2728 loss = 54.0270
   1  270 / 2728 loss = 64.2407
   1  280 / 2728 loss = 62.8281
   1  290 / 2728 loss = 

 33%|███▎      | 1/3 [03:52<07:44, 232.06s/it]

2024-12-13 03:16:21: epcoh =    1 , loss = 3632.2472 , time = 230.02 s
^_^-training data finished-^_^
   2    0 / 2728 loss = 12.5466
   2   10 / 2728 loss = 26.6247
   2   20 / 2728 loss = 38.4835
   2   30 / 2728 loss = 24.6686
   2   40 / 2728 loss = 24.0222
   2   50 / 2728 loss = 25.2037
   2   60 / 2728 loss = 59.3731
   2   70 / 2728 loss = 18.6037
   2   80 / 2728 loss = 25.0113
   2   90 / 2728 loss = 11.1854
   2  100 / 2728 loss = 20.9979
   2  110 / 2728 loss = 20.1346
   2  120 / 2728 loss = 16.9313
   2  130 / 2728 loss = 31.0983
   2  140 / 2728 loss = 25.6598
   2  150 / 2728 loss = 22.0393
   2  160 / 2728 loss = 16.7425
   2  170 / 2728 loss = 46.6616
   2  180 / 2728 loss = 12.4737
   2  190 / 2728 loss = 19.2847
   2  200 / 2728 loss = 12.4561
   2  210 / 2728 loss = 20.7970
   2  220 / 2728 loss = 15.7941
   2  230 / 2728 loss = 13.9002
   2  240 / 2728 loss = 29.9998
   2  250 / 2728 loss = 15.8242
   2  260 / 2728 loss = 23.9696
   2  270 / 2728 loss = 20.4426
  

 67%|██████▋   | 2/3 [07:44<03:52, 232.21s/it]

2024-12-13 03:20:14: epcoh =    2 , loss = 1201.3587 , time = 230.24 s
^_^-training data finished-^_^
   3    0 / 2728 loss = 7.0562
   3   10 / 2728 loss = 6.7474
   3   20 / 2728 loss = 24.2776
   3   30 / 2728 loss = 7.0304
   3   40 / 2728 loss = 30.9595
   3   50 / 2728 loss = 7.0629
   3   60 / 2728 loss = 18.1968
   3   70 / 2728 loss = 20.9187
   3   80 / 2728 loss = 16.8881
   3   90 / 2728 loss = 12.8894
   3  100 / 2728 loss = 7.5952
   3  110 / 2728 loss = 26.7025
   3  120 / 2728 loss = 14.4435
   3  130 / 2728 loss = 5.9817
   3  140 / 2728 loss = 45.7971
   3  150 / 2728 loss = 10.6404
   3  160 / 2728 loss = 10.6670
   3  170 / 2728 loss = 19.6354
   3  180 / 2728 loss = 11.8884
   3  190 / 2728 loss = 6.1379
   3  200 / 2728 loss = 12.7986
   3  210 / 2728 loss = 5.9122
   3  220 / 2728 loss = 12.5729
   3  230 / 2728 loss = 30.5421
   3  240 / 2728 loss = 5.7372
   3  250 / 2728 loss = 11.4766
   3  260 / 2728 loss = 5.1758
   3  270 / 2728 loss = 5.0418
   3  280 / 2

100%|██████████| 3/3 [11:36<00:00, 232.26s/it]

2024-12-13 03:24:06: epcoh =    3 , loss = 891.0544 , time = 230.38 s
